<a href="https://colab.research.google.com/github/dpostolovski/eeg_is/blob/train_compare_full_data/NAJDOLGIOT_KOD_SO_POSTOI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [34]:
#@title Инсталирање и вчитување на потребните библиотеки

# Библиотека за истражување, визуелизација и анализирање на човечки 
# неврофизиолошки податоци (EEG, sEEG и др)
!pip install mne 
!pip install termcolor
!mkdir saved_models

%tensorflow_version 1.12.0

mkdir: cannot create directory ‘saved_models’: File exists
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.12.0`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
from termcolor import colored
import mne
from sklearn.decomposition import PCA, FastICA
import mne
from sklearn import metrics 
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ModelCheckpoint 
from sklearn.model_selection import train_test_split
import scipy.io as sio
from scipy.fft import fft
from scipy import signal
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras import backend as K
from tensorflow import keras
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.optimizers import Adam
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas.util.testing as tm

import sys
sys.path.append('drive/My Drive/Интелигентни Системи')
from EEGModels import DeepConvNet, EEGNet

K.set_image_data_format('channels_first')

In [36]:
df=pd.read_csv('drive/My Drive/Интелигентни Системи/output_format.csv')
#df.iat[0,0] = 5
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
1,1,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
2,1,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
3,2,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
4,2,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
5,2,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
6,3,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
7,3,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
8,3,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
9,4,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN


In [37]:
# Иницијализација на променливите каде ќе бидат вчитани податоците
data = []
labels = []
events = []
targets = []

for i in range(1, 2): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model1 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model1.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model1.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model1.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 1 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.3168 - acc: 0.1600
Epoch 00001: val_loss improved from inf to 2.17509, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 3s 892us/sample - loss: 2.3041 - acc: 0.1622 - val_loss: 2.1751 - val_acc: 0.1942
Epoch 2/400
3500/3600 [============================>.] - ETA: 0s - loss: 2.1270 - acc: 0.1854
Epoch 00002: val_loss did not improve from 2.17509
3600/3600 [==============================] - 1s 187us/sample - loss: 2.1265 - acc: 0.1844 - val_loss: 2.2666 - val_acc: 0.2025
Epoch 3/400
3500/3600 [============================>.] - ETA: 0s - loss: 2.1223 - acc: 0.20

In [0]:
def reshape_data_to_mne_format(data):
  mne_array = np.swapaxes(data, 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0], 1, 8, 350)

  return mne_array

In [39]:
# Иницијализација на променливите каде ќе бидат вчитани тест податоците
test_data = []
test_events = []
test_runs_per_block = [[i for i in range(3)] for j in range(15)] # Covek, Sesija

for i in range(1, 2): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model1.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session,block+2] = prediction +1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (1) ======================
Вчитување тест податоци од испитниот примерок 1...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 1 се вчитани.

SBJ00| Test_data: (8, 350, 8000)
SBJ00| Test_events: 8000
SBJ00 / S00| Runs per block: 6
SBJ00 / S01| Runs per block: 7
SBJ00 / S02| Runs per block: 7
SBJ00| Predictions: 8000
============== Сесија (0) ==============
6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,
============== Сесија (1) ==============
6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,
============== Сесија (2) ==============
3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,
Stigna li do kraj: True
====================== Примерок (1) ======================




In [40]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
4,2,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
5,2,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
6,3,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
7,3,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
8,3,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
9,4,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN


In [41]:
for i in range(2, 3): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model2 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model2.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model2.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model2.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 2 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.3099 - acc: 0.1527
Epoch 00001: val_loss improved from inf to 2.33148, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 4s 1ms/sample - loss: 2.3074 - acc: 0.1542 - val_loss: 2.3315 - val_acc: 0.1558
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.1672 - acc: 0.1803
Epoch 00002: val_loss improved from 2.33148 to 2.07297, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 207us/sample - loss: 2.1648 - acc: 0.1867 - val_loss: 2.0730 - val_acc: 0.2333
Epoch 3/400
3500/3600 [========

In [42]:
for i in range(2, 3): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model2.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+3,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (2) ======================
Вчитување тест податоци од испитниот примерок 2...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 2 се вчитани.

SBJ01| Test_data: (8, 350, 8800)
SBJ01| Test_events: 8800
SBJ01 / S00| Runs per block: 9
SBJ01 / S01| Runs per block: 8
SBJ01 / S02| Runs per block: 5
SBJ01| Predictions: 8800
============== Сесија (0) ==============
8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,
============== Сесија (1) ==============
2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,
============== Сесија (2) ==============
2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,
Stigna li do kraj: True
====================== Примерок (2) ======================




In [43]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
7,3,2,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
8,3,3,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN
9,4,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN


In [44]:
for i in range(3, 4): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model3 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model3.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model3.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model3.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 3 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3500/3600 [============================>.] - ETA: 0s - loss: 2.3234 - acc: 0.1663
Epoch 00001: val_loss improved from inf to 2.11654, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 4s 1ms/sample - loss: 2.3212 - acc: 0.1661 - val_loss: 2.1165 - val_acc: 0.1992
Epoch 2/400
3500/3600 [============================>.] - ETA: 0s - loss: 2.1766 - acc: 0.1866
Epoch 00002: val_loss did not improve from 2.11654
3600/3600 [==============================] - 1s 191us/sample - loss: 2.1824 - acc: 0.1858 - val_loss: 2.2188 - val_acc: 0.1733
Epoch 3/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.1105 - acc: 0.2232

In [45]:
for i in range(3, 4): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model3.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+6,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (3) ======================
Вчитување тест податоци од испитниот примерок 3...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 3 се вчитани.

SBJ02| Test_data: (8, 350, 7600)
SBJ02| Test_events: 7600
SBJ02 / S00| Runs per block: 6
SBJ02 / S01| Runs per block: 6
SBJ02 / S02| Runs per block: 7
SBJ02| Predictions: 7600
============== Сесија (0) ==============
3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,
============== Сесија (1) ==============
6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,
============== Сесија (2) ==============
6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,
Stigna li do kraj: True
====================== Примерок (3) ======================




In [46]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,NaN


In [47]:
for i in range(4, 5): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model4 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model4.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model4.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model4.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 4 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.4462 - acc: 0.1491
Epoch 00001: val_loss improved from inf to 2.52206, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 4s 1ms/sample - loss: 2.4321 - acc: 0.1494 - val_loss: 2.5221 - val_acc: 0.1792
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.2292 - acc: 0.1742
Epoch 00002: val_loss improved from 2.52206 to 2.35845, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 205us/sample - loss: 2.2261 - acc: 0.1736 - val_loss: 2.3585 - val_acc: 0.1142
Epoch 3/400
3500/3600 [========

In [48]:
for i in range(4, 5): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model4.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+9,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (4) ======================
Вчитување тест податоци од испитниот примерок 4...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 4 се вчитани.

SBJ03| Test_data: (8, 350, 6800)
SBJ03| Test_events: 6800
SBJ03 / S00| Runs per block: 5
SBJ03 / S01| Runs per block: 5
SBJ03 / S02| Runs per block: 7
SBJ03| Predictions: 6800
============== Сесија (0) ==============
1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,
============== Сесија (1) ==============
7,7,7,7,6,6,7,5,7,8,7,5,7,6,6,3,1,6,6,6,5,5,5,5,5,8,6,5,6,5,6,7,4,7,6,6,5,6,5,6,6,7,1,5,5,5,5,5,2,6,
============== Сесија (2) ==============
7,3,7,3,6,6,7,5,6,1,6,6,6,7,6,6,6,6,6,7,7,4,4,7,6,7,6,6,7,4,7,7,1,6,6,6,1,1,1,1,6,6,6,1,4,4,4,6,6,6,
Stigna li do kraj: True
====================== Примерок (4) ======================




In [49]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [50]:
for i in range(5, 6): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model5 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model5.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model5.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model5.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 5 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.3944 - acc: 0.1579
Epoch 00001: val_loss improved from inf to 2.15632, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 5s 1ms/sample - loss: 2.3860 - acc: 0.1589 - val_loss: 2.1563 - val_acc: 0.1475
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.2740 - acc: 0.1579
Epoch 00002: val_loss did not improve from 2.15632
3600/3600 [==============================] - 1s 190us/sample - loss: 2.2708 - acc: 0.1600 - val_loss: 2.6325 - val_acc: 0.0908
Epoch 3/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2063 - acc: 0.1718

In [51]:
for i in range(5, 6): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model5.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+12,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (5) ======================
Вчитување тест податоци од испитниот примерок 5...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 5 се вчитани.

SBJ04| Test_data: (8, 350, 8800)
SBJ04| Test_events: 8800
SBJ04 / S00| Runs per block: 8
SBJ04 / S01| Runs per block: 7
SBJ04 / S02| Runs per block: 7
SBJ04| Predictions: 8800
============== Сесија (0) ==============
5,4,4,4,6,4,5,4,2,6,3,6,3,4,3,5,4,2,1,4,1,1,3,6,3,6,4,1,4,6,2,4,6,2,4,4,6,3,3,4,1,3,3,1,4,4,1,3,3,5,
============== Сесија (1) ==============
3,6,3,5,2,6,7,7,6,6,6,4,6,6,6,6,6,6,6,6,4,6,6,6,4,6,6,2,6,4,6,1,1,6,4,6,4,6,4,6,4,4,6,6,6,6,6,6,6,4,
============== Сесија (2) ==============
4,3,3,6,5,7,6,4,6,4,4,6,6,6,3,2,6,5,2,5,5,2,6,2,7,6,4,4,4,4,5,5,2,2,2,6,1,7,1,6,6,7,7,2,2,7,5,6,4,2,
Stigna li do kraj: True
====================== Примерок (5) ======================




In [52]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [53]:
for i in range(6, 7): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model6 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model6.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model6.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model6.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 6 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3200/3600 [=========================>....] - ETA: 0s - loss: 2.3842 - acc: 0.1406
Epoch 00001: val_loss improved from inf to 2.12568, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 5s 1ms/sample - loss: 2.3824 - acc: 0.1422 - val_loss: 2.1257 - val_acc: 0.1825
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.2966 - acc: 0.1642
Epoch 00002: val_loss did not improve from 2.12568
3600/3600 [==============================] - 1s 188us/sample - loss: 2.2969 - acc: 0.1636 - val_loss: 2.2256 - val_acc: 0.1467
Epoch 3/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.2152 - acc: 0.1764

In [54]:
for i in range(6, 7): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model6.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+15,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (6) ======================
Вчитување тест податоци од испитниот примерок 6...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 6 се вчитани.

SBJ05| Test_data: (8, 350, 8000)
SBJ05| Test_events: 8000
SBJ05 / S00| Runs per block: 4
SBJ05 / S01| Runs per block: 7
SBJ05 / S02| Runs per block: 9
SBJ05| Predictions: 8000
============== Сесија (0) ==============
1,3,1,8,3,3,3,4,7,2,5,3,5,3,3,2,4,3,8,5,4,2,4,3,2,8,2,4,3,3,3,8,3,3,3,3,3,7,8,5,3,4,4,3,7,2,7,2,5,8,
============== Сесија (1) ==============
3,4,1,7,1,1,1,1,4,1,3,3,1,5,4,8,3,2,8,3,4,3,4,3,8,4,3,3,3,3,2,3,4,8,3,4,3,5,5,4,2,5,5,5,5,5,5,5,5,5,
============== Сесија (2) ==============
2,3,2,5,3,3,3,5,2,3,2,1,3,3,3,3,2,3,3,5,1,3,3,1,1,3,3,3,3,3,5,3,1,1,3,3,3,3,3,1,1,3,2,3,3,3,1,1,3,3,
Stigna li do kraj: True
====================== Примерок (6) ======================




In [55]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [56]:
for i in range(7, 8): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model7 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model7.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model7.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model7.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 7 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3500/3600 [============================>.] - ETA: 0s - loss: 2.4373 - acc: 0.1386
Epoch 00001: val_loss improved from inf to 2.23719, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 6s 2ms/sample - loss: 2.4329 - acc: 0.1381 - val_loss: 2.2372 - val_acc: 0.1475
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.2315 - acc: 0.1576
Epoch 00002: val_loss did not improve from 2.23719
3600/3600 [==============================] - 1s 189us/sample - loss: 2.2405 - acc: 0.1542 - val_loss: 2.4563 - val_acc: 0.1125
Epoch 3/400
3200/3600 [=========================>....] - ETA: 0s - loss: 2.1692 - acc: 0.1759

In [57]:
for i in range(7, 8): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model7.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+18,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (7) ======================
Вчитување тест податоци од испитниот примерок 7...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 7 се вчитани.

SBJ06| Test_data: (8, 350, 9600)
SBJ06| Test_events: 9600
SBJ06 / S00| Runs per block: 9
SBJ06 / S01| Runs per block: 5
SBJ06 / S02| Runs per block: 10
SBJ06| Predictions: 9600
============== Сесија (0) ==============
4,7,5,7,4,5,4,1,1,1,1,6,2,1,5,6,2,1,7,5,5,5,5,4,1,5,2,8,8,4,5,4,5,1,5,4,4,5,1,4,5,5,5,1,1,4,4,4,1,4,
============== Сесија (1) ==============
2,2,2,8,2,5,2,8,2,5,8,5,1,1,8,2,1,1,1,7,1,7,3,1,5,1,8,2,1,1,7,5,8,1,2,2,2,5,2,1,2,1,5,5,5,2,2,2,2,2,
============== Сесија (2) ==============
2,7,7,5,7,5,4,3,3,3,5,5,5,5,5,5,3,8,5,5,1,7,3,4,4,2,5,2,5,3,5,5,5,3,1,5,5,1,3,3,8,5,1,3,5,5,3,5,5,1,
Stigna li do kraj: True
====================== Примерок (7) ======================




In [58]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [59]:
for i in range(8, 9): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model8 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model8.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model8.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model8.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 8 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2870 - acc: 0.1900
Epoch 00001: val_loss improved from inf to 2.23248, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 6s 2ms/sample - loss: 2.2769 - acc: 0.1931 - val_loss: 2.2325 - val_acc: 0.2108
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.0936 - acc: 0.2427
Epoch 00002: val_loss did not improve from 2.23248
3600/3600 [==============================] - 1s 187us/sample - loss: 2.1054 - acc: 0.2408 - val_loss: 3.0164 - val_acc: 0.0958
Epoch 3/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.0408 - acc: 0.2797

In [60]:
for i in range(8, 9): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model8.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+21,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (8) ======================
Вчитување тест податоци од испитниот примерок 8...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 8 се вчитани.

SBJ07| Test_data: (8, 350, 7600)
SBJ07| Test_events: 7600
SBJ07 / S00| Runs per block: 8
SBJ07 / S01| Runs per block: 7
SBJ07 / S02| Runs per block: 4
SBJ07| Predictions: 7600
============== Сесија (0) ==============
5,8,2,8,2,2,1,1,1,8,5,2,5,2,1,2,5,7,1,8,1,1,1,3,3,7,1,2,5,8,7,7,1,1,2,5,2,4,1,5,7,5,1,8,2,2,5,5,2,8,
============== Сесија (1) ==============
2,7,1,5,1,2,1,1,2,7,7,7,1,7,7,7,7,7,7,3,1,2,2,2,7,7,1,6,1,8,2,1,5,2,7,6,1,8,1,7,6,5,7,1,7,8,2,1,7,7,
============== Сесија (2) ==============
4,1,6,6,1,2,1,1,1,1,1,1,1,1,5,3,3,1,1,1,1,5,3,3,1,1,1,1,1,1,1,7,7,1,1,1,7,7,1,1,1,1,1,8,1,7,1,1,7,1,
Stigna li do kraj: True
====================== Примерок (8) ======================




In [61]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [62]:
for i in range(9, 10): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model9 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model9.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model9.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model9.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 9 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.1180 - acc: 0.2318
Epoch 00001: val_loss improved from inf to 2.16939, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 7s 2ms/sample - loss: 2.1121 - acc: 0.2297 - val_loss: 2.1694 - val_acc: 0.2000
Epoch 2/400
3500/3600 [============================>.] - ETA: 0s - loss: 1.8936 - acc: 0.2657
Epoch 00002: val_loss improved from 2.16939 to 1.84940, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 209us/sample - loss: 1.8887 - acc: 0.2653 - val_loss: 1.8494 - val_acc: 0.2392
Epoch 3/400
3500/3600 [========

In [63]:
for i in range(9, 10): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model9.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+24,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (9) ======================
Вчитување тест податоци од испитниот примерок 9...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 9 се вчитани.

SBJ08| Test_data: (8, 350, 10000)
SBJ08| Test_events: 10000
SBJ08 / S00| Runs per block: 8
SBJ08 / S01| Runs per block: 8
SBJ08 / S02| Runs per block: 9
SBJ08| Predictions: 10000
============== Сесија (0) ==============
6,5,6,6,7,6,8,6,8,6,4,8,6,6,5,6,5,8,6,8,8,8,6,6,6,6,6,6,6,5,5,5,6,6,5,6,8,5,5,8,6,6,6,6,6,5,5,5,3,5,
============== Сесија (1) ==============
3,4,8,4,1,6,1,6,3,3,6,3,3,6,6,1,1,1,1,1,1,2,1,4,2,1,2,1,1,3,1,4,8,1,1,1,1,4,1,4,1,4,1,1,1,1,1,3,6,6,
============== Сесија (2) ==============
5,2,2,2,2,3,2,3,2,2,3,3,2,2,2,2,3,3,2,2,6,4,2,2,4,6,2,1,1,3,1,2,3,3,4,2,3,2,3,2,3,3,2,2,2,3,2,2,4,2,
Stigna li do kraj: True
====================== Примерок (9) ======================




In [64]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [65]:
for i in range(10, 11): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model10 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model10.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model10.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model10.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 10 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.3195 - acc: 0.1771
Epoch 00001: val_loss improved from inf to 2.12739, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 7s 2ms/sample - loss: 2.3164 - acc: 0.1756 - val_loss: 2.1274 - val_acc: 0.1808
Epoch 2/400
3500/3600 [============================>.] - ETA: 0s - loss: 2.2226 - acc: 0.1951
Epoch 00002: val_loss did not improve from 2.12739
3600/3600 [==============================] - 1s 190us/sample - loss: 2.2206 - acc: 0.1964 - val_loss: 2.2004 - val_acc: 0.2200
Epoch 3/400
3200/3600 [=========================>....] - ETA: 0s - loss: 2.1261 - acc: 0.228

In [66]:
for i in range(10, 11): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model10.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+27,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (10) ======================
Вчитување тест податоци од испитниот примерок 10...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 10 се вчитани.

SBJ09| Test_data: (8, 350, 6400)
SBJ09| Test_events: 6400
SBJ09 / S00| Runs per block: 5
SBJ09 / S01| Runs per block: 5
SBJ09 / S02| Runs per block: 6
SBJ09| Predictions: 6400
============== Сесија (0) ==============
1,1,8,7,3,1,1,8,1,5,1,1,7,7,6,1,1,6,4,6,1,1,1,5,1,1,5,1,1,5,1,1,1,1,1,5,1,6,2,6,2,5,5,1,1,1,1,5,1,5,
============== Сесија (1) ==============
1,5,5,6,1,1,1,1,5,6,1,5,6,1,2,5,8,4,6,6,1,1,1,1,1,1,2,6,5,8,6,1,1,8,1,1,5,8,6,2,1,8,5,5,6,6,2,1,2,1,
============== Сесија (2) ==============
5,8,5,5,5,6,5,2,5,5,5,2,6,6,2,5,5,5,1,5,5,5,1,1,6,2,6,6,2,5,5,5,5,5,5,5,1,6,5,5,5,5,5,5,2,5,5,5,5,5,
Stigna li do kraj: True
====================== Примерок (10) ======================




In [67]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [68]:
for i in range(11, 12): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model11 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model11.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model11.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model11.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 11 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2707 - acc: 0.1618
Epoch 00001: val_loss improved from inf to 2.15439, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 8s 2ms/sample - loss: 2.2699 - acc: 0.1600 - val_loss: 2.1544 - val_acc: 0.1583
Epoch 2/400
3500/3600 [============================>.] - ETA: 0s - loss: 2.1430 - acc: 0.1911
Epoch 00002: val_loss improved from 2.15439 to 2.03312, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 210us/sample - loss: 2.1464 - acc: 0.1900 - val_loss: 2.0331 - val_acc: 0.2075
Epoch 3/400
3200/3600 [=======

In [69]:
for i in range(11, 12): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model11.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+30,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (11) ======================
Вчитување тест податоци од испитниот примерок 11...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 11 се вчитани.

SBJ10| Test_data: (8, 350, 8000)
SBJ10| Test_events: 8000
SBJ10 / S00| Runs per block: 8
SBJ10 / S01| Runs per block: 4
SBJ10 / S02| Runs per block: 8
SBJ10| Predictions: 8000
============== Сесија (0) ==============
3,4,4,4,3,4,3,8,4,3,3,2,4,4,1,3,3,3,3,3,1,4,3,1,1,3,4,4,4,4,3,4,4,2,1,3,3,4,7,4,4,3,5,4,3,1,3,4,4,4,
============== Сесија (1) ==============
3,1,1,4,8,4,4,2,8,8,3,3,1,4,4,3,1,1,7,2,1,1,7,1,3,3,8,8,1,3,3,3,7,3,3,7,3,3,3,7,2,3,1,7,3,8,2,3,8,3,
============== Сесија (2) ==============
4,2,4,4,4,4,4,4,8,8,8,8,8,8,4,8,8,4,4,4,8,4,8,8,3,1,4,4,6,2,4,4,8,4,4,8,4,1,7,7,4,4,8,4,4,7,8,4,8,4,
Stigna li do kraj: True
====================== Примерок (11) ======================




In [70]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [71]:
for i in range(12, 13): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model12 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model12.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model12.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model12.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 12 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2845 - acc: 0.1826
Epoch 00001: val_loss improved from inf to 2.53376, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 8s 2ms/sample - loss: 2.2707 - acc: 0.1847 - val_loss: 2.5338 - val_acc: 0.1150
Epoch 2/400
3200/3600 [=========================>....] - ETA: 0s - loss: 2.0837 - acc: 0.2300
Epoch 00002: val_loss did not improve from 2.53376
3600/3600 [==============================] - 1s 192us/sample - loss: 2.0852 - acc: 0.2317 - val_loss: 2.7278 - val_acc: 0.1258
Epoch 3/400
3400/3600 [===========================>..] - ETA: 0s - loss: 1.9880 - acc: 0.288

In [72]:
for i in range(12, 13): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model12.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+33,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (12) ======================
Вчитување тест податоци од испитниот примерок 12...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 12 се вчитани.

SBJ11| Test_data: (8, 350, 8800)
SBJ11| Test_events: 8800
SBJ11 / S00| Runs per block: 8
SBJ11 / S01| Runs per block: 9
SBJ11 / S02| Runs per block: 5
SBJ11| Predictions: 8800
============== Сесија (0) ==============
1,1,1,1,1,1,6,3,6,1,1,6,1,6,3,1,3,7,2,2,3,5,5,2,6,1,3,7,7,7,6,6,1,3,6,5,7,3,2,5,1,3,1,6,7,3,1,7,7,3,
============== Сесија (1) ==============
3,2,6,2,2,3,2,5,7,7,7,3,1,1,3,7,3,1,7,5,7,7,3,7,7,1,2,1,6,5,3,1,1,2,5,8,3,5,5,7,1,3,5,1,2,1,5,5,2,5,
============== Сесија (2) ==============
1,6,2,2,1,2,2,2,2,2,2,2,2,2,6,2,3,2,5,2,2,6,2,1,2,5,2,6,6,1,1,2,2,4,1,2,2,2,6,3,6,5,5,6,2,1,2,2,2,2,
Stigna li do kraj: True
====================== Примерок (12) ======================




In [73]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [74]:
for i in range(13, 14): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model13 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model13.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model13.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model13.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 13 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2161 - acc: 0.2059
Epoch 00001: val_loss improved from inf to 1.91527, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 9s 3ms/sample - loss: 2.1991 - acc: 0.2100 - val_loss: 1.9153 - val_acc: 0.2700
Epoch 2/400
3400/3600 [===========================>..] - ETA: 0s - loss: 1.8254 - acc: 0.2800
Epoch 00002: val_loss improved from 1.91527 to 1.70765, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 217us/sample - loss: 1.8227 - acc: 0.2789 - val_loss: 1.7077 - val_acc: 0.2942
Epoch 3/400
3400/3600 [=======

In [75]:
for i in range(13, 14): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model13.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+36,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (13) ======================
Вчитување тест податоци од испитниот примерок 13...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 13 се вчитани.

SBJ12| Test_data: (8, 350, 10000)
SBJ12| Test_events: 10000
SBJ12 / S00| Runs per block: 9
SBJ12 / S01| Runs per block: 8
SBJ12 / S02| Runs per block: 8
SBJ12| Predictions: 10000
============== Сесија (0) ==============
6,6,1,5,3,1,1,3,5,6,6,6,6,6,6,1,1,1,1,1,4,1,1,5,1,1,1,4,4,1,6,3,4,4,1,6,1,5,1,1,1,4,1,1,1,5,6,1,1,1,
============== Сесија (1) ==============
6,1,5,4,1,1,6,6,1,6,8,6,1,1,6,1,6,1,5,6,1,1,1,1,1,1,1,6,1,1,5,6,7,1,4,1,4,5,6,4,1,6,7,6,8,6,1,6,6,1,
============== Сесија (2) ==============
1,6,5,1,5,2,6,7,6,6,2,1,1,1,4,4,1,6,1,5,5,2,6,4,1,5,1,6,1,6,7,7,1,6,6,1,6,4,2,6,6,6,7,1,1,7,2,2,6,1,
Stigna li do kraj: True
====================== Примерок (13) ======================




In [76]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [77]:
for i in range(14, 15): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model14 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model14.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model14.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model14.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 14 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2940 - acc: 0.1565
Epoch 00001: val_loss improved from inf to 2.20905, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 10s 3ms/sample - loss: 2.2871 - acc: 0.1561 - val_loss: 2.2091 - val_acc: 0.1417
Epoch 2/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.2158 - acc: 0.1797
Epoch 00002: val_loss improved from 2.20905 to 2.19250, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 223us/sample - loss: 2.2197 - acc: 0.1783 - val_loss: 2.1925 - val_acc: 0.1342
Epoch 3/400
3400/3600 [======

In [78]:
for i in range(14, 15): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model14.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+39,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (14) ======================
Вчитување тест податоци од испитниот примерок 14...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 14 се вчитани.

SBJ13| Test_data: (8, 350, 9200)
SBJ13| Test_events: 9200
SBJ13 / S00| Runs per block: 10
SBJ13 / S01| Runs per block: 6
SBJ13 / S02| Runs per block: 7
SBJ13| Predictions: 9200
============== Сесија (0) ==============
4,4,3,4,3,8,2,4,3,1,4,4,4,4,4,4,4,4,4,7,4,4,4,4,4,4,3,4,4,4,4,4,3,4,4,4,6,4,4,4,3,6,4,6,3,6,4,3,4,1,
============== Сесија (1) ==============
2,3,4,3,3,3,3,3,3,3,3,3,2,3,2,3,3,7,3,3,2,3,7,8,4,5,2,4,3,3,2,3,3,2,3,2,2,3,3,2,2,2,2,3,3,2,3,3,3,2,
============== Сесија (2) ==============
7,4,6,4,6,3,7,6,4,6,4,6,4,4,6,4,7,4,4,4,4,4,4,5,4,4,4,4,4,4,4,4,4,4,4,4,4,7,3,4,7,4,4,4,4,4,4,4,4,7,
Stigna li do kraj: True
====================== Примерок (14) ======================




In [79]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [80]:
for i in range(15, 16): # Итерација низ секој испитен примерок
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_data = np.empty(0)
  temp_labels = np.empty(0)
  temp_events = np.empty(0)
  temp_targets = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainData.mat"
    temp = loadmat(full_path)['trainData']
    if temp_data.size != 0:
      temp_data = np.concatenate((temp_data, temp), axis=2)
    else: 
      temp_data = np.array(temp)

    # Вчитување на label-ите
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainLabels.txt"
    with open(full_path, "r") as file_labels:
      temp = file_labels.read().splitlines()
      temp = np.repeat(temp, 8*10)
      if temp_labels.size != 0:
        temp_labels = np.concatenate((temp_labels, temp))
      else:
        temp_labels = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_events.size != 0:
        temp_events = np.append(temp_events, temp)
      else:
        temp_events = np.array(temp)
      

    # Вчитување на редоследот на објекти кои се target
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_train_set + "/Train/trainTargets.txt"
    with open(full_path, "r") as file_targets:
      temp = file_targets.read().splitlines()
      if temp_targets.size != 0:
        temp_targets = np.concatenate((temp_targets, temp))
      else:
        temp_targets = np.array(temp)
    print("\t - Податоците од сесија " + str(j) + " се вчитани.")

  for j in range(4, 8): # Итерација низ секоја сесија
    file_train_set = 'S' + format(j, '02') 

      
  # Зачувај ги податоците вчитани од испитниот примерок во низа
  data.append(temp_data)
  labels.append(temp_labels)
  events.append(temp_events)
  targets.append(temp_targets)

  
  print("Податоците од испитниот примерок " + str(i) + " се вчитани.")


  #data = target_events_data_scaled
  mne_array = np.swapaxes(data[i-1], 0, 2) # (епохa, канал, настан). 
  mne_array = np.swapaxes(mne_array, 1, 2) # (епохa, канал, настан). 
  mne_array = mne_array.reshape(mne_array.shape[0],1, 8,350)
  print(mne_array.shape)

  events_arr = events[i-1].astype(np.int)
  labels_arr = labels[i-1].astype(np.int)


  X_train, X_test, y_train, y_test = train_test_split(mne_array, labels_arr-1, test_size=0.25, random_state=42)


  model15 = DeepConvNet(nb_classes = 8, Chans = 8, Samples = 350)
  model15.compile(loss = 'categorical_crossentropy', metrics=['accuracy'],optimizer = Adam(0.0009))
  checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                                verbose=1, save_best_only=True)


  #clf = RandomForestClassifier(max_depth=5)
  #clf.fit(X_train, y_train)
  #score = clf.score(X_test, y_test)
  # print(score)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

  num_batch_size=100
  num_epochs=400
  model15.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, \
            validation_data=(X_test, y_test),callbacks=[checkpointer], verbose=1)

  score = model15.evaluate(X_test, y_test, verbose=1)
  print(score)

	 - Податоците од сесија 1 се вчитани.
	 - Податоците од сесија 2 се вчитани.
	 - Податоците од сесија 3 се вчитани.
Податоците од испитниот примерок 15 се вчитани.
(4800, 1, 8, 350)
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Version 10
Train on 3600 samples, validate on 1200 samples
Epoch 1/400
3400/3600 [===========================>..] - ETA: 0s - loss: 2.3338 - acc: 0.1532
Epoch 00001: val_loss improved from inf to 2.31488, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 10s 3ms/sample - loss: 2.3257 - acc: 0.1550 - val_loss: 2.3149 - val_acc: 0.1467
Epoch 2/400
3300/3600 [==========================>...] - ETA: 0s - loss: 2.2481 - acc: 0.1667
Epoch 00002: val_loss improved from 2.31488 to 2.23936, saving model to saved_models/weights.best.basic_mlp.hdf5
3600/3600 [==============================] - 1s 214us/sample - loss: 2.2556 - acc: 0.1644 - val_loss: 2.2394 - val_acc: 0.1733
Epoch 3/400
3500/3600 [======

In [81]:
for i in range(15, 16): # Итерација низ секој испитен примерок
  print(f"====================== Примерок ({i}) ======================")
  print("Вчитување тест податоци од испитниот примерок " + str(i) + "...")
  
  file_name = 'SBJ' + format(i, '02')
  
  # Иницијализација на помошни променливи
  temp_test_data = np.empty(0)
  temp_test_events = np.empty(0)
  
  for j in range(1, 4): # Итерација низ секоја сесија
    file_test_set = 'S' + format(j, '02') + '/Test'

    # Вчитување на податоците
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testData.mat"
    temp = loadmat(full_path)['testData']
    if temp_test_data.size != 0:
      temp_test_data = np.concatenate((temp_test_data, temp), axis=2)
    else: 
      temp_test_data = np.array(temp)

    # Вчитување на редоследот на светкање
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/testEvents.txt"
    with open(full_path, "r") as file_events:
      temp = file_events.read().splitlines()
      if temp_test_events.size != 0:
        temp_test_events = np.append(temp_test_events, temp)
      else:
        temp_test_events = np.array(temp)

    # Вчитување на бројот на runs 
    full_path = 'drive/My Drive/Интелигентни Системи/Data/' + file_name + "/" + file_test_set + "/runs_per_block.txt"
    with open(full_path, "r") as runs_per_block:
      test_runs_per_block[i-1][j-1] = int(runs_per_block.read())

    print("\t - Тест податоците од сесија " + str(j) + " се вчитани.")
  # Зачувај ги тест податоците вчитани од испитниот примерок во низа
  test_data.append(temp_test_data)
  test_events.append(temp_test_events)
  print("Тест податоците од испитниот примерок " + str(i) + " се вчитани.\n")

  print("SBJ" + str(format(i-1, '02')) + "| Test_data: " + str(test_data[i-1].shape)) # test_data to predict
  print("SBJ" + str(format(i-1, '02')) + "| Test_events: " + str(len(test_events[i-1]))) # test_events
  for j in range (1,4):
    print("SBJ" + str(format(i-1, '02')) + " / S" + str(format(j-1, '02')) + "| Runs per block: " + str(test_runs_per_block[i-1][j-1])) # runs per block in SJB01, SJ00 

  to_predict_data = reshape_data_to_mne_format(test_data[i-1])
  predictions = model15.predict(to_predict_data)
  print("SBJ" + str(format(i-1, '02')) + "| Predictions: " + str(len(predictions)))
  # np.savetxt("predictions.csv", predictions, delimiter=",")


  # ========= FALI USTE DA SE ISPARSIRA PREDICTIONOT... NE E SREDEN OVOJ KOD DOLE =======

  int_pred = np.argmax(predictions, axis=1)
  int_ytest = np.argmax(y_test, axis=1)

  session_start = 0
  start_prediction_index = 0
  end_prediction_index = 0
  for session in range(0, 3):
    print(f"============== Сесија ({session}) ==============")
    for block in range(0, 50):    
      events_per_block = test_runs_per_block[i-1][session]

      start_prediction_index = session_start + (block*events_per_block)*8
      end_prediction_index = session_start + ((block+1)*events_per_block)*8

      block_prediction = int_pred[start_prediction_index:end_prediction_index]
      prediction = np.bincount(block_prediction).argmax()
      df.iat[session+42,block+2] = prediction+1
      # UNCOMMENT ZA PODOBAR PRIKAZ :)
      # print(f"Session {session} | Block: {block} | Prediction: {prediction} | Address: {end_prediction_index}")

      print(str(prediction+1) + ",", end="")
    session_start = end_prediction_index
    print("")
  print("Stigna li do kraj: " + str(session_start == len(predictions)))
  print(f"====================== Примерок ({i}) ======================\n\n")

====================== Примерок (15) ======================
Вчитување тест податоци од испитниот примерок 15...
	 - Тест податоците од сесија 1 се вчитани.
	 - Тест податоците од сесија 2 се вчитани.
	 - Тест податоците од сесија 3 се вчитани.
Тест податоците од испитниот примерок 15 се вчитани.

SBJ14| Test_data: (8, 350, 8800)
SBJ14| Test_events: 8800
SBJ14 / S00| Runs per block: 6
SBJ14 / S01| Runs per block: 6
SBJ14 / S02| Runs per block: 10
SBJ14| Predictions: 8800
============== Сесија (0) ==============
4,7,4,3,1,1,5,1,5,5,5,2,8,1,5,4,1,5,1,5,1,1,3,5,5,5,5,5,5,5,6,5,1,3,5,1,1,1,5,5,1,1,1,5,5,5,5,1,1,1,
============== Сесија (1) ==============
7,6,4,5,5,5,5,5,5,5,5,4,5,7,5,5,5,5,5,1,5,5,7,7,7,7,7,5,7,5,5,5,7,5,7,3,4,7,7,7,7,5,2,5,4,5,7,5,5,8,
============== Сесија (2) ==============
5,4,8,4,2,4,1,4,5,5,1,4,1,1,4,5,2,5,5,4,2,1,1,2,5,5,5,2,1,4,4,4,1,4,4,5,1,7,1,4,4,7,3,2,1,5,4,1,5,5,
Stigna li do kraj: True
====================== Примерок (15) ======================




In [82]:
df

,SUBJECT,SESSION,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,Unnamed: 52
0,1,1,6,6,3,6,6,3,6,6,6,6,6,6,7,6,5,6,4,3,3,7,7,8,3,3,7,6,6,4,8,6,7,6,6,4,7,7,7,7,5,3,3,2,6,4,8,3,4,5,5,7,NaN
1,1,2,6,3,2,1,2,1,3,6,2,2,6,2,7,1,2,3,7,1,3,1,6,6,2,2,6,1,6,6,1,2,2,3,2,6,2,2,2,5,6,2,6,6,6,2,2,2,3,6,6,2,NaN
2,1,3,3,3,3,3,3,3,3,3,7,3,7,3,3,3,6,6,6,6,3,3,3,3,7,3,6,3,3,6,3,3,3,3,6,6,6,6,6,6,6,3,3,6,3,3,6,3,6,7,1,6,NaN
3,2,1,8,8,8,8,8,8,8,8,8,8,8,8,8,3,8,8,8,8,8,8,8,8,8,8,8,7,8,8,3,8,5,3,8,8,7,8,8,7,8,8,8,8,8,4,8,4,8,7,8,8,NaN
4,2,2,2,7,6,6,6,6,7,6,2,6,6,6,6,2,6,2,2,6,2,6,6,7,2,6,6,6,2,2,2,5,6,2,2,6,5,2,2,2,6,6,6,7,2,6,6,2,6,6,2,2,NaN
5,2,3,2,7,2,6,7,4,2,6,6,1,6,7,2,6,6,2,2,2,6,2,6,2,2,7,7,2,7,7,7,7,1,7,7,2,7,2,2,7,2,6,2,6,2,6,2,2,7,2,2,2,NaN
6,3,1,3,3,4,4,4,3,6,3,3,5,4,7,4,3,3,5,6,3,3,4,5,3,3,4,5,5,3,4,3,3,4,4,3,3,3,4,4,3,3,4,3,4,4,6,3,3,4,3,4,4,NaN
7,3,2,6,1,7,7,7,7,7,6,7,4,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,1,7,6,6,6,6,6,6,1,6,7,7,6,6,7,6,7,1,6,6,6,6,6,NaN
8,3,3,6,4,8,8,5,7,8,8,8,2,8,8,8,8,4,4,2,8,4,8,4,7,4,8,4,4,4,4,4,8,1,8,4,4,4,4,4,8,4,4,2,5,4,2,8,2,2,2,2,8,NaN
9,4,1,1,1,2,1,5,5,6,5,5,5,6,5,1,1,6,1,1,1,6,1,5,6,6,5,2,6,1,5,6,6,2,1,1,1,6,2,6,2,6,1,1,1,1,1,7,1,6,6,6,6,NaN


In [0]:
df.to_csv('drive/My Drive/Интелигентни Системи/output.csv', index = False)